# Optimization Engine

## Notebook Guide

- **Purpose:** Run dispatch optimization and visualize cost/carbon tradeoffs.
- **Inputs:** configs/optimization.yaml
- **Outputs:** dispatch and SOC plots.
- **Run:** Execute cells top‑to‑bottom. If a file is missing, run the earlier pipeline notebook first.


## Dispatch Optimization (LP)

This notebook demonstrates the cost + carbon objective with battery/grid constraints.

In [ ]:
from pathlib import Path
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print('repo_root:', repo_root)

In [ ]:
import numpy as np
from gridpulse.optimizer.lp_dispatch import optimize_dispatch

cfg_path = repo_root / 'configs' / 'optimization.yaml'
cfg = {}
if cfg_path.exists():
    import yaml
    cfg = yaml.safe_load(cfg_path.read_text())

horizon = 24
load = np.full(horizon, 8000.0)
renew = np.full(horizon, 3200.0)

plan = optimize_dispatch(load, renew, cfg)
plan.keys()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({
    'grid_mw': plan['grid_mw'],
    'battery_charge_mw': plan['battery_charge_mw'],
    'battery_discharge_mw': plan['battery_discharge_mw'],
    'soc_mwh': plan['soc_mwh'],
})

fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
df[['grid_mw','battery_charge_mw','battery_discharge_mw']].plot(ax=ax[0], title='Dispatch')
df[['soc_mwh']].plot(ax=ax[1], title='State of Charge')
plt.tight_layout()